## 文件处理预备知识
- ### [open函数](https://www.runoob.com/python/python-func-open.html)
- ### [with as语法](https://www.jianshu.com/p/c00df845323c)

## [pandas五分钟快速入门](https://www.runoob.com/pandas/pandas-dataframe.html) 

In [5]:
import os
os.makedirs(os.path.join('.', 'data'), exist_ok=True) # 在本目录下创建文件夹data(可递归创建) exist_ok：只有在目录不存在时创建目录，目录已存在时不会抛出异常。
data_file = os.path.join('.', 'data', 'house_tiny.csv') # 数据存储的文件名
with open(data_file, 'w') as f:  # 创建一个写入流f 如果该文件已存在则打开文件，并从开头开始编辑，即原有内容会被删除。如果该文件不存在，创建新文件。
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')


## 使用pandas读取`csv`文件

In [7]:
import pandas as pd
data = pd.read_csv(data_file)
data
# 数据每行描述了房间数量、巷子类型和房屋价格

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


### 处理缺失值
上例中NAN表示缺失的值
> 为了处理缺失的数据，典型的方法包括插值和删除，其中插值用替代值代替缺失值。而删除则忽略缺失值。在这里，我们将考虑插值,通过位置索引iloc，我们将data分成inputs和outputs，其中前者为data的前两列，而后者为data的最后一列。对于inputs中缺少的数值，我们用同一列的均值替换“NaN”项。

In [16]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2] #需要使用pandas内的iloc来将数据切片
print(type(data))
inputs = inputs.fillna(inputs.mean()) # 按列取mean 并填充至该列的nan值
print(inputs)


<class 'pandas.core.frame.DataFrame'>
NumRooms    3.0
dtype: float64
   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


> 对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。由于“巷子”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”，pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。 

In [9]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)


   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


## 数据预处理之最后一步 转为张量

In [10]:
import torch

X, Y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, Y # x即为输入数据 y为输出数据 一行代表一个样本


(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))